In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'pandas'

In [31]:
# In this assignment, you will predict the life expectancy of a newborn based on several attributes (features) related to the region which he/she was born in.

In [32]:
# Import Dataset
life_expectancy_train_dataset = "s3://rmit-ml-2020s1-lab-data/Assignment1_data/train.csv"
life_expectancy_test_dataset = pd.read_csv("s3://rmit-ml-2020s1-lab-data/Assignment1_data/train.csv")"

In [33]:
# Set Dataframes
le_train_DF = pd.read_csv(life_expectancy_train_dataset)

NameError: name 'pd' is not defined

In [34]:
# Analysis of Data

# 1) The data has many outliers, and useless variables
#    that do not contribute to the target value (life expectancy)
# 2) Only a few variables within the dataset have been found to have a correlation with our target value: TARGET_LifeExpectancy

# Box Plot to find the outliers within the dataset
le_train_DF.plot(kind="box", subplots=True, layout=(6,6),
                sharex=False, sharey=False, figsize=(50,50))


NameError: name 'le_train_DF' is not defined

In [35]:
# Analysis- Correlation Heat Map
import seaborn as sns

correlations = le_train_DF.corr()

f, ax = plt.subplots(figsize=(11, 9))

sns.heatmap(round(correlations,2), ax=ax, cmap="coolwarm", vmin=-1, vmax=1, fmt='.2f', linewidths=.05)
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [36]:
# Normalise the data, using mean normalisation method
# Through the analysis, it has been observed on the box plots that there are significant outliers.
# Min Max Scaling does not handle outliers very well, so mean normalisation has been chosen instead
independent_scaler = preprocessing.StandardScaler()
target_scaler = preprocessing.StandardScaler()

# Set X as all independent variables (minus TARGET_LifeExpectancy)
X_train_DF = le_train_DF.iloc[:, 2:]
X_test_DF = le_test_DF.iloc[:, 1:]

# Set Y as dependent variable 'TARGET_LifeExpectancy'
Y_train_DF = le_train_DF['TARGET_LifeExpectancy']

# Convert into Array to manipulate
X_train_values = X_train_DF.values
Y_train_values = Y_train_DF.values
X_test_values = X_test_DF.values

X_train_values_scaled = independent_scaler.fit_transform(X_train_values)
Y_train_values_scaled = target_scaler.fit_transform(Y_train_values.reshape(-1, 1))
X_test_values_scaled = independent_scaler.fit_transform(X_test_values)

X_train = pd.DataFrame(X_train_values_scaled, columns=X_train_DF.columns)
Y_train = pd.DataFrame(Y_train_values_scaled)
X_test = pd.DataFrame(X_test_values_scaled, columns=X_test_DF.columns)

NameError: name 'preprocessing' is not defined

In [37]:
from sklearn import model_selection
number_splits = 10
kFold = model_selection.KFold(n_splits=number_splits, shuffle=True)

ModuleNotFoundError: No module named 'sklearn'

In [38]:
lRegPara = np.linspace(0.001,0.5,num=10)
lRegPara

NameError: name 'np' is not defined

In [39]:
# Splitting Training and Validation set, via KFold Method
# To determine the best regression parameter is best, utilise cross validation to obtain the most effective one.
# Utilising Lasso Regression, although similar to Ridge Regression, it is better at eliminating use variables within our model
lRegPara = np.linspace(0.0001,0.5,num=100)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
lResults = np.zeros((number_splits,len(lRegPara)))
lResults_1 = np.zeros((number_splits,len(lRegPara)))
nsplit = 0
polyFeat = PolynomialFeatures(degree=3, include_bias=True)
model_scores = []

for trainIndex, validIndex in kFold.split(X_train):
    # Get the training and validation data
    trainX = np.array(X_train.loc[trainIndex])
    trainY = np.array(Y_train.loc[trainIndex])
    validX = np.array(X_train.loc[validIndex])
    validY = np.array(Y_train.loc[validIndex])
    
    # This is where you're polynomial model is used!
    polyFitTrainX = polyFeat.fit_transform(trainX)
    polyFitValidX = polyFeat.fit_transform(validX)

    for j, regPara in enumerate(lRegPara):
        polyLassoReg = linear_model.Lasso(alpha=regPara, normalize=False, tol=0.1)
        model = polyLassoReg.fit(polyFitTrainX, trainY)
        predY = polyLassoReg.predict(polyFitValidX)
        r2 = r2_score(validY, predY, multioutput='raw_values')
        model_scores.append((j, regPara))
        lResults[nsplit, j] = (r2)
        
    plt.plot(lRegPara, lResults[nsplit, :], label='Fold '+str(nsplit+1))
    nsplit = nsplit + 1
    
plt.xlabel("Alpha")
plt.ylabel("R-Score")
plt.legend()

plt.figure()
plt.errorbar(lRegPara, np.mean(lResults,axis=0), yerr=np.std(lResults,axis=0),capsize=3)
plt.xlabel("Alpha")
plt.ylabel("Average R-Score")

NameError: name 'np' is not defined

In [40]:
# print(model_scores)
# a = np.max(lResults)
# print(np.where(lResults == a))
# print(lResults[7, 2])

for trainIndex, validIndex in kFold.split(X_train):
    # Get the training and validation data
    trainX = np.array(X_test.loc[trainIndex])
    trainY = np.array(Y_train.loc[trainIndex])
    validX = np.array(X_test.loc[validIndex])
    validY = np.array(Y_train.loc[validIndex])
    
    # This is where you're polynomial model is used!
    polyFitTrainX = polyFeat.fit_transform(trainX)
    polyFitValidX = polyFeat.fit_transform(validX)
    
polyLassoReg = linear_model.Lasso(alpha=0.010198989898989898, normalize=False, tol=0.1)
model = polyLassoReg.fit(polyFitTrainX, trainY)
predY = polyLassoReg.predict(polyFitValidX)
r2_score(validY, predY, multioutput='raw_values')

NameError: name 'pd' is not defined